In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from astropy.table import Table, vstack
import pandas as pd
import glob as glob

size_outside_cut = 100
muon_efficiency_cut = 1
min_impact = 2.2199933748101555
max_impact = 9.983608702234397

color_dict = {
    'muon_sim_00025deg': 'purple',
    'muon_sim_00046deg': 'orange',
    'muon_sim_00092deg': 'green',
    'muon_sim_0015deg': 'red',
    'muon_sim_002deg': 'blue',
    'muon_from_proton_sim_00046deg' : 'lightgreen',
    'muon_from_proton_sim_001deg' : 'blue',
    'winter_data_2024': 'black'
}

fmt_dict = {
    'muon_sim_00025deg': 'o',
    'muon_sim_00046deg': 'o',
    'muon_sim_00092deg': 'o',
    'muon_sim_0015deg': 'o',
    'muon_sim_002deg': 'o',
    'muon_from_proton_sim_00046deg' : 'o',
    'muon_from_proton_sim_001deg' : 'o',
    'winter_data_2024': 'x'   
}

ls_dict = {
    'muon_sim_00025deg': '--',
    'muon_sim_00046deg': '--',
    'muon_sim_00092deg': '--',
    'muon_sim_0015deg': '--',
    'muon_sim_002deg': '--',
    'muon_from_proton_sim_00046deg' : '--',
    'muon_from_proton_sim_001deg' : '--',
    'winter_data_2024': '-'
} 


# Methods

In [ ]:
def reweight_gaug(data, zenith, alpha  = 0.7, is_data=False):
    if is_data:
        weights = np.ones(len(data))/len(data)
    else:
        zenith = np.deg2rad(zenith)
        weights = np.ones(len(data))
        weights = data['mc_energy']**(-alpha) * (1/(1 + 1.1 * data['mc_energy'] * 1000 * np.cos(np.deg2rad(zenith))/115) + 0.054/(1 + 1.1 * data['mc_energy'] * 1000 * np.cos(np.deg2rad(zenith))/850))
    return weights / np.sum(weights)
    
def read_and_process_fits(files):
    dat = None
    for muon_file in files:
        dat2 = Table.read(muon_file, format='fits')
        
        # Convert columns to boolean if they exist
        for col in ['good_ring', 'is_valid', 'parameters_at_limit']:
            if col in dat2.colnames:
                dat2[col] = dat2[col].astype(bool)
        
        # Stack the tables
        dat = vstack([dat, dat2]) if dat is not None else dat2
    
    return dat

def filter_dataframe(df, muon_efficiency_cut, size_outside_cut):
    return df[
        df['good_ring'] & 
        (df['muon_efficiency'] < muon_efficiency_cut) & 
        (df['size_outside'] < size_outside_cut)
    ]

def calculate_survival_percentage(df, df_cut):
    return (len(df_cut) / len(df)) * 100

def apply_radius_cut(df, min_radius_cut, max_radius_cut):
    return df[
        (df['ring_radius'] > min_radius_cut) &
        (df['ring_radius'] < max_radius_cut)
    ]
    
def apply_cuts(df, completeness_cut, containment_cut, min_radius_cut, max_radius_cut, center_distance_cut, impact_distance_cut, num_pixels_in_ring_cut, quantile_cut=1):
    """ Apply cuts to the dataframe """
    
    #ring_width_quantile = df['ring_width'].quantile(quantile_cut)
    
    return df[
        (df['ring_completeness'] > completeness_cut) &
        (df['ring_containment'] > containment_cut) &
        (df['ring_radius'] > min_radius_cut) &
        (df['ring_radius'] < max_radius_cut) &
        (df['ring_center_x'].abs() < center_distance_cut) &
        (df['ring_center_y'].abs() < center_distance_cut) &
        (df['impact_parameter'] < impact_distance_cut) &
        (df['num_pixels_in_ring'] < num_pixels_in_ring_cut) &
        (df['ring_width'] < quantile_cut)  # Apply the cut on ring_width
    ]


def plot_columns(df_data, df_sim, df_sim_protons, columns, rows, cols, apply_weight=True, df_name='None', plot_option=True, percentile=False):
    fig, axes = plt.subplots(rows, cols, figsize=(10, 13))
    axes = axes.flatten()
    data_weights = reweight_gaug(df_data, 10, 10, True)
    sim_weights = reweight_gaug(df_sim, 10, 0.7, False)
    sim_protons_weights = df_sim_protons['mc_energy']**(-0.71)
    
    for i, col in enumerate(columns):
        data_values = df_data[col]
        sim_values = df_sim[col]
        sim_protons_values = df_sim_protons[col]

        combined_values = pd.concat([data_values, sim_values, sim_protons_values])

        min_value = combined_values.min()
        max_value = combined_values.max()

        total_events = len(combined_values)
        num_bins = int(max(len(df_data)**0.3, len(df_sim))**0.3)
        num_bins = max(10, min(num_bins, 50))
        #num_bins = int(min(len(df_data)**0.5, len(df_sim))**0.5)
        #num_bins = max(10, min(num_bins, 50))

        if col == 'ring_width':
            if percentile:
                min_value = combined_values.min()
                max_value = 0.07
                axes[i].set_xlim(0, 0.08)
                num_bins = int(min(len(df_data), len(df_data))**0.4)
                bins = np.linspace(min_value, max_value, num_bins + 1)
            else:
                min_value = combined_values.min()
                max_value = 0.15
                axes[i].set_xlim(0, 0.15)
                num_bins = int(max(len(df_data)**0.3, len(df_sim))**0.3)
                bins = np.linspace(min_value, max_value, num_bins + 1)
        
        bins = np.linspace(min_value, max_value, num_bins + 1)
        if apply_weight:
            axes[i].hist(data_values, weights=data_weights, bins=bins, alpha=0.6, color='k', density=True, label='Data', histtype='step', lw=2)
            axes[i].hist(sim_values, weights=sim_weights, bins=bins, alpha=0.85, color='orange', density=True, label=f'Muon Simulation 0.0046deg', histtype='step', lw=2)
            axes[i].hist(sim_protons_values, weights=sim_protons_weights, bins=bins, alpha=0.85, color='lightgreen', density=True, label='Muon From Proton Simulations 0.0046deg', histtype='step', lw=2)
            #axes[i].hist(sim_protons_values, weights=sim_protons_weights, bins=bins, alpha=0.85, color='red', density=True, label='Muon Simulations 0.015 deg', histtype='step', lw=2)
        else:
            axes[i].hist(data_values, bins=bins, alpha=0.6, color='k', density=True, label='Data', histtype='step', lw=2)
            axes[i].hist(sim_values, bins=bins, alpha=0.85, color='orange', density=True, label=f'Muon Simulations 0.0046 deg', histtype='step', lw=2)
            axes[i].hist(sim_protons_values, bins=bins, alpha=0.85, color='lightgreen', density=True, label='Muon From Proton Simulations 0.0046deg', histtype='step', lw=2)
            #axes[i].hist(sim_protons_values, bins=bins, alpha=0.85, color='red', density=True, label='Muon Simulations 0.015 deg', histtype='step', lw=2)

        axes[i].set_title(f'Histogram of {col}')
        axes[i].set_xlabel(col)
        axes[i].set_ylabel('Normalized Frequency')
        axes[i].legend(loc='lower left')

    plt.tight_layout()
    if plot_option:
        plt.savefig(f'/Users/vdk/muons2024/basic_distributions/PROTONS_noringcenter_{name}_radiuscut={min_radius_cut}-{max_radius_cut}_completeness={ring_completeness_cut}_centerdistance={ring_center_distance_cut}_impactdistance={impact_distance_cut}_widthquantile={QUANTILE_CUT}.pdf', dpi=200)
    plt.show()


In [ ]:
listdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_tuned_reflectivity_00025alignment/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_00025_align = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)
df_sim_00025_align.name = 'muon_sim_00025deg'

listdir1 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_tuned_reflectivity/tables/*')
listdir2 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/for_paper/nsbtune2024year_no_outliers_additional_true/tables/*')
dat = read_and_process_fits(listdir1 + listdir2)
df = dat.to_pandas()
df_sim_00046_align = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)
df_sim_00046_align.name = 'muon_sim_00046deg'

listdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/psf_sim/nsbtune2024year_tuned_reflectivity_00092alignment/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_00092_align = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)
df_sim_00092_align.name = 'muon_sim_00092deg'

listdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_tuned_reflectivity_0015alignment/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_0015_align = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)
df_sim_0015_align.name = 'muon_sim_0015deg'

listdir= glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/psf_sim/nsbtune2024year_tuned_reflectivity_002alignment/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_002_align = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)
df_sim_002_align.name = 'muon_sim_002deg'

In [ ]:
listdir= glob.glob('/Users/vdk/muons2024/psf_work/proton_simulation/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_protons = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)
df_sim_protons.name = 'protons_oiginal'

listdir = glob.glob('/Users/vdk/muons2024/psf_work/proton_simulation/more_runs_/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_protons_more = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)
df_sim_protons_more.name = 'protons'

listdir = glob.glob('/Users/vdk/muons2024/psf_work/proton_simulation/more_runs_2/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_protons_more_2 = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)
df_sim_protons_more_2.name = 'protons'

listdir = glob.glob('/Users/vdk/muons2024/psf_work/proton_simulation/more_runs_3/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_protons_more_3 = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)
df_sim_protons_more_3.name = 'protons'

listdir = glob.glob('/Users/vdk/muons2024/psf_work/proton_simulation/more_runs_4/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_protons_more_4 = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)
df_sim_protons_more_4.name = 'protons'

listdir = glob.glob('/Users/vdk/muons2024/psf_work/proton_simulation/more_runs_6/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_protons_more_6 = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)
df_sim_protons_more_6.name = 'protons'

listdir = glob.glob('/Users/vdk/muons2024/psf_work/proton_simulation/more_runs_7/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_protons_more_7 = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)
df_sim_protons_more_7.name = 'protons'

listdir = glob.glob('/Users/vdk/muons2024/psf_work/proton_simulation/more_runs_9/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_protons_more_9 = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)
df_sim_protons_more_9.name = 'protons'
df_sim_protons_more_9

df_sim_protons_combined = pd.concat([df_sim_protons, df_sim_protons_more, df_sim_protons_more_2, df_sim_protons_more_3, df_sim_protons_more_4, df_sim_protons_more_6, df_sim_protons_more_7, df_sim_protons_more_9], ignore_index=True)
df_sim_protons_combined.name = 'muon_from_proton_sim_00046deg'

listdir = glob.glob('/Users/vdk/muons2024/psf_work/proton_simulation/more_runs_3_001align/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_protons_more_3_001 = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)
df_sim_protons_more_3_001.name = 'protons_001'

listdir = glob.glob('/Users/vdk/muons2024/psf_work/proton_simulation/more_runs_4_001align/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_protons_more_4_001 = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)
df_sim_protons_more_4_001.name = 'protons_001'

df_sim_protons_combined_001 = pd.concat([df_sim_protons_more_3_001, df_sim_protons_more_4_001], ignore_index=True)
df_sim_protons_combined_001.name = 'muon_from_proton_sim_001deg'

In [ ]:
df_sim_protons_combined

In [ ]:
listdir = glob.glob('/Users/vdk/muons2024/psf_work/no_integration_cor/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_no_int_cor = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)
df_no_int_cor.name = 'no_integr_cor'

# Data

In [ ]:
listdir = glob.glob('/Users/vdk/muons2024/data_quality_winter_2024/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_winter_data = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)
df_winter_data.name = 'winter_data_2024'

In [ ]:
QUANTILE_CUT = 0.2
width_data_quantile_cut = df_winter_data['ring_width'].quantile(QUANTILE_CUT)

ring_completeness_cut = 0.6
ring_containment_cut = 0
num_pixels_in_ring_cut = 1000
# Cut on ring radius coming from the energy range in simulations (8.4 GeV - 1 TeV)
min_radius_cut = 1.003
max_radius_cut = 1.235

# Cut on ring center distance comes from the narrow cone of simulated muons, viewcone = 0.9 degrees
ring_center_distance_cut = 0.65

# Muons were simulated to be randomly scattered in the circle with radius of 9.8 meters
impact_distance_cut = 5.5

df_list = [df_sim_00025_align, df_sim_00046_align, df_sim_00092_align, df_sim_0015_align, df_sim_002_align, df_sim_protons_combined, df_sim_protons_combined_001, df_winter_data]
cut_datasets = {}
data_width_binned_dict = {}

bin_edges = max_radius_cut - min_radius_cut
bin_bins = np.linspace(min_radius_cut, max_radius_cut, 10)
bin_centers = (bin_bins[1:] + bin_bins[:-1]) / 2

for df in df_list:
    
    cut_datasets[df.name] = apply_cuts(
        df, 
        ring_completeness_cut, 
        ring_containment_cut, 
        min_radius_cut, 
        max_radius_cut, 
        ring_center_distance_cut, 
        impact_distance_cut,
        num_pixels_in_ring_cut,
        width_data_quantile_cut
    ).copy()

    cut_datasets[df.name] = cut_datasets[df.name].assign(
        radius_bin=pd.cut(
            cut_datasets[df.name]['ring_radius'], 
            bins=bin_bins, 
            include_lowest=True
        )
    )
    data_width_binned_dict[df.name] = cut_datasets[df.name].groupby('radius_bin', observed=False)['ring_width'].agg(['mean', 'sum', 'count', 'std']).reset_index()
    data_width_binned_dict[df.name]['error'] = data_width_binned_dict[df.name]['std'] / np.sqrt(data_width_binned_dict[df.name]['count'])
    print(f"Events survived for {df.name}: {len(cut_datasets[df.name])} ({calculate_survival_percentage(df, cut_datasets[df.name]):.2f}%)")

df_to_plot = {
    'muon_sim_00025deg': False,
    'muon_sim_00046deg': True,
    'muon_sim_00092deg': True,
    'muon_sim_0015deg': True,
    'muon_sim_002deg': False,
    'muon_from_proton_sim_00046deg': True,
    'muon_from_proton_sim_001deg': False,
    'winter_data_2024': True
}

plt.figure(figsize=(10, 6.5))

for name, df in data_width_binned_dict.items():
    if df_to_plot[name]:
        plt.errorbar(
            bin_centers,
            df['mean'] / bin_centers,
            yerr=df['error'] / bin_centers,
            fmt=fmt_dict[name],
            label=name,
            color=color_dict[name],
            capsize=7,
            markersize=10,
            alpha=0.9
        )
    
        slope, intercept = np.polyfit(bin_centers, df['mean'] / bin_centers, 1)
        y_fit = slope * bin_centers + intercept
        plt.plot(bin_centers, y_fit, color=color_dict[name], linestyle=ls_dict[name])#, label=f'{name} fit')

plt.legend()

# Customize the plot
plt.xlabel('Ring Radius (bin centers)')
plt.ylabel('Mean Ring Width / Radius')
plt.ylim(0.02, 0.09)
plt.title(f'min/max radiuscut = {min_radius_cut}/{max_radius_cut},completeness={ring_completeness_cut},centerdistance<{ring_center_distance_cut},impactdistance<{impact_distance_cut},widthquantile={QUANTILE_CUT}') 
plt.grid(alpha=0.3)
plt.tight_layout()  # Adjust layout for better spacing
plt.savefig(f'/Users/vdk/muons2024/basic_distributions/relative_broadening/PROTONS_radiuscut={min_radius_cut}-{max_radius_cut}_completeness={ring_completeness_cut}_center distance={ring_center_distance_cut}_impact distance={impact_distance_cut}_width quantile={QUANTILE_CUT}.pdf', dpi=200)
plt.show()


In [ ]:
width_data_quantile_cut

# Basic distributions

In [ ]:
#name_of_columns = ['ring_size', 'ring_radius', 'ring_center_x', 'ring_center_y', 'num_pixels_in_ring', 'mean_pixel_charge_around_ring', 'radial_stdev', 'ring_width']
name_of_columns = ['ring_size', 'ring_radius', 'num_pixels_in_ring', 'mean_pixel_charge_around_ring', 'radial_stdev', 'ring_width']

print('====================================================')
print('====================================================')
print('\n')
print('-----------> Data + Simulation for mirror alignment', name[-20:], 'deg <--------------\n')
print(f'min/max radius cut = {min_radius_cut}/{max_radius_cut}, \ncompleteness>{ring_completeness_cut} \ncenter distance<{ring_center_distance_cut},\nimpact distance<{impact_distance_cut}, \nwidth quantile={QUANTILE_CUT}')
print('\n')
print('====================================================')
print('====================================================')
percentile_bool = True
if QUANTILE_CUT == 1:
    percentile_bool = False
plot_columns(cut_datasets['winter_data_2024'], cut_datasets['muon_sim_00046deg'], cut_datasets['muon_from_proton_sim_00046deg'], name_of_columns, rows = 3, cols = 2, df_name = name, percentile=percentile_bool, plot_option=True)

In [ ]:
max(cut_datasets['muon_from_proton_sim_00046deg']['impact_y_array'])

In [ ]:
max(cut_datasets['muon_sim_00046deg']['ring_center_x'])

In [ ]:
cut_datasets['muon_from_proton_sim_00046deg'].keys()

In [ ]:
plt.figure(figsize=(10, 6.5))
z,x,c = plt.hist(cut_datasets['winter_data_2024']['size_outside'][cut_datasets['winter_data_2024']['size_outside'] >= 0.0000], log=True, bins=np.linspace(0,500,30), alpha=0.6, color='k', density=True, label='Data', histtype='step', lw=2)
z,x,c = plt.hist(cut_datasets['muon_sim_00046deg']['size_outside'][cut_datasets['muon_sim_00046deg']['size_outside'] >= 0.00000], log=True, bins=np.linspace(0,500,30), alpha=0.85, color='orange', density=True, label='Muon Simulations 0.0046 deg', histtype='step', lw=2)
z,x,c = plt.hist(cut_datasets['muon_from_proton_sim_00046deg']['size_outside'][cut_datasets['muon_from_proton_sim_00046deg']['size_outside'] >= 0.00000], log=True, bins=np.linspace(0,500,30), alpha=0.85, color='lightgreen', density=True, label='Muon From Proton Simulations 0.0046deg', histtype='step', lw=2)
#plt.xlim(0,20)
plt.legend()

In [ ]:
z,x,c = plt.hist(cut_datasets['winter_data_2024']['size_outside'][cut_datasets['winter_data_2024']['size_outside'] > 0.0000001], log=True, bins=30, alpha=0.6, color='k', density=True, label='Data', histtype='step', lw=2)
z,x,c = plt.hist(cut_datasets['muon_sim_00046deg']['size_outside'][cut_datasets['muon_sim_00046deg']['size_outside'] > 0.0000001], log=True, bins=30, alpha=0.85, color='orange', density=True, label='Muon Simulations 0.0046 deg', histtype='step', lw=2)
z,x,c = plt.hist(cut_datasets['muon_from_proton_sim_00046deg']['size_outside'][cut_datasets['muon_from_proton_sim_00046deg']['size_outside'] > 0.0000001], log=True, bins=30, alpha=0.85, color='lightgreen', density=True, label='Muon From Proton Simulations 0.0046deg', histtype='step', lw=2)
#plt.xlim()
plt.legend()

In [ ]:
print(f"Fraction of size outside pixels in data {round(len(cut_datasets['winter_data_2024']['size_outside'][cut_datasets['winter_data_2024']['size_outside'] > 0.0000001]) / len(cut_datasets['winter_data_2024']['size_outside'][cut_datasets['winter_data_2024']['size_outside'] < 0.0000001]) * 100, 2)}%")
print(f"Fraction of size outside pixels in muon sim {round(len(cut_datasets['muon_sim_00046deg']['size_outside'][cut_datasets['muon_sim_00046deg']['size_outside'] > 0.0000001]) / len(cut_datasets['muon_sim_00046deg']['size_outside'][cut_datasets['muon_sim_00046deg']['size_outside'] < 0.0000001]) * 100, 2)}%")
print(f"Fraction of size outside pixels in proton sim {round(len(cut_datasets['muon_from_proton_sim_00046deg']['size_outside'][cut_datasets['muon_from_proton_sim_00046deg']['size_outside'] > 0.0000001]) / len(cut_datasets['muon_from_proton_sim_00046deg']['size_outside'][cut_datasets['muon_from_proton_sim_00046deg']['size_outside'] < 0.0000001]) * 100, 2)}%")

In [ ]:
z,x,c = plt.hist(cut_datasets['winter_data_2024']['mean_pixel_charge_around_ring'][cut_datasets['winter_data_2024']['size_outside'] > 0.0000001], log=False, bins=20, alpha=0.6, color='k', density=True, label='Data', histtype='step', lw=2)
z,x,c = plt.hist(cut_datasets['muon_sim_00046deg']['mean_pixel_charge_around_ring'][cut_datasets['muon_sim_00046deg']['size_outside'] > 0.0000001], log=False, bins=20, alpha=0.85, color='orange', density=True, label='Muon Simulations 0.0046 deg', histtype='step', lw=2)
z,x,c = plt.hist(cut_datasets['muon_from_proton_sim_00046deg']['mean_pixel_charge_around_ring'][cut_datasets['muon_from_proton_sim_00046deg']['size_outside'] > 0.0000001], log=False, bins=20, alpha=0.85, color='lightgreen', density=True, label='Muon From Proton Simulations 0.0046deg', histtype='step', lw=2)
plt.xlim(-0.5, 1.5)
plt.legend()

In [ ]:
z,x,c = plt.hist(cut_datasets['winter_data_2024']['mean_pixel_charge_around_ring'][cut_datasets['winter_data_2024']['size_outside'] < 0.0000001], log=False, bins=20, alpha=0.6, color='k', density=True, label='Data', histtype='step', lw=2)
z,x,c = plt.hist(cut_datasets['muon_sim_00046deg']['mean_pixel_charge_around_ring'][cut_datasets['muon_sim_00046deg']['size_outside'] < 0.0000001], log=False, bins=20, alpha=0.85, color='orange', density=True, label='Muon Simulations 0.0046 deg', histtype='step', lw=2)
z,x,c = plt.hist(cut_datasets['muon_from_proton_sim_00046deg']['mean_pixel_charge_around_ring'][cut_datasets['muon_from_proton_sim_00046deg']['size_outside'] < 0.0000001], log=False, bins=20, alpha=0.85, color='lightgreen', density=True, label='Muon From Proton Simulations 0.0046deg', histtype='step', lw=2)
plt.xlim(-1, 1)
plt.legend()

In [ ]:
z,x,c = plt.hist(cut_datasets['winter_data_2024']['mean_pixel_charge_around_ring'][cut_datasets['winter_data_2024']['size_outside'] >= 0], log=True, bins=20, alpha=0.6, color='k', density=True, label='Data', histtype='step', lw=2)
z,x,c = plt.hist(cut_datasets['muon_sim_00046deg']['mean_pixel_charge_around_ring'][cut_datasets['muon_sim_00046deg']['size_outside'] >= 0], log=True, bins=20, alpha=0.85, color='orange', density=True, label='Muon Simulations 0.0046 deg', histtype='step', lw=2)
z,x,c = plt.hist(cut_datasets['muon_from_proton_sim_00046deg']['mean_pixel_charge_around_ring'][cut_datasets['muon_from_proton_sim_00046deg']['size_outside'] >= 0], log=True, bins=20, alpha=0.85, color='lightgreen', density=True, label='Muon From Proton Simulations 0.0046deg', histtype='step', lw=2)
plt.xlim(-1, 4)
plt.legend()

In [ ]:
plt.figure(figsize=(10, 6.5))
z,x,c = plt.hist(cut_datasets['winter_data_2024']['mean_pixel_charge_around_ring'], log=False, bins=40, alpha=0.6, color='k', density=True, label='Data', histtype='step', lw=2)
z,x,c = plt.hist(cut_datasets['muon_sim_00046deg']['mean_pixel_charge_around_ring'], log=False, bins=40, alpha=0.85, color='orange', density=True, label='Muon Simulations 0.0046 deg', histtype='step', lw=2)
z,x,c = plt.hist(cut_datasets['muon_sim_0015deg']['mean_pixel_charge_around_ring'], log=False, bins=40, alpha=0.85, color='purple', density=True, label='Muon Simulations 0.015 deg', histtype='step', lw=2)
z,x,c = plt.hist(cut_datasets['muon_from_proton_sim_00046deg']['mean_pixel_charge_around_ring'], log=False, bins=40, alpha=0.85, color='lightgreen', density=True, label='Muon From Proton Simulations 0.0046deg', histtype='step', lw=2)
z,x,c = plt.hist(df_no_int_cor['mean_pixel_charge_around_ring'], log=False, bins=40, alpha=0.85, color='red', density=True, label='No integration correction', histtype='step', lw=2)
plt.xlim(-0.5, 1)
plt.legend()

In [ ]:
listdir = glob.glob('/Users/vdk/muons2024/psf_work/proton_simulation/more_runs_8/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_protons_more_8 = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)
df_sim_protons_more_8.name = 'protons_008'

In [ ]:
len(df_sim_protons_more_7['mc_energy'])

In [ ]:
plt.figure(figsize=(10, 6.5))
z,x,c = plt.hist(df_sim_protons_combined['mc_energy'], bins=np.linspace(0.01,1,100), alpha=0.6, color='k', density=True, label='Proton simulation', histtype='step', lw=2,log=True)
plt.legend()

In [ ]:
plt.figure(figsize=(10, 6.5))
z,x,c = plt.hist(df_sim_protons_more_7['mc_energy'][df_sim_protons_more_7['mc_energy'] < 1], bins=np.linspace(0.01,1,100), alpha=0.6, color='k', density=True, label='Proton simulation', histtype='step', lw=2,log=True)
z,x,c = plt.hist(df_sim_00046_align['mc_energy'], bins=np.linspace(0.01,1,100), alpha=0.6, color='red', density=True, label='Muon simulation', histtype='step', lw=2,log=True)
plt.legend()

In [ ]:
# Calculate weights
weights_protons = reweight_gaug(df_sim_protons_more_7, 10, False)
weights_muons = reweight_gaug(df_sim_00046_align, 10, False)

# Plot histograms with weights
plt.figure(figsize=(12, 8.5))
z, x, c = plt.hist(df_sim_protons_more_7['mc_energy'][(df_sim_protons_more_7['mc_energy'] < 1) & (df_sim_protons_more_7['mc_energy'] > 0.1)], bins=np.linspace(0.01, 1, 100), alpha=0.6, color='k', density=True, label='Proton simulation', histtype='step', lw=2, log=True)
z, x, c = plt.hist(df_sim_00046_align['mc_energy'][df_sim_00046_align['mc_energy'] > 0.1], bins=np.linspace(0.01, 1, 100), alpha=0.6, color='red', density=True, label='Muon simulation', histtype='step', lw=2, log=True)
plt.legend()
plt.xlabel('TeV')
plt.show()

In [ ]:
mc_energies_1 = df_sim_protons_combined['mc_energy']
mc_energies_2 = df_sim_00046_align['mc_energy']

len(mc_energies_2)
mc_energies_1.min()

In [ ]:
df_sim_protons_combined['mc_energy'].min()
df_sim_00046_align['mc_energy'].min()


In [ ]:
from scipy.optimize import curve_fit

import numpy as np
import matplotlib.pyplot as plt

# Assume 'data' is your DataFrame containing MC events
#mc_energies_1 = df_sim_protons_combined['mc_energy'][(df_sim_protons_combined['mc_energy'] < 1) & (df_sim_protons_combined['mc_energy'] > 0.01)]
mc_energies_1 = df_sim_protons_combined['mc_energy']
mc_energies_2 = df_sim_00046_align['mc_energy'][:len(mc_energies_1)]

# Define energy bins (adjust as needed)
bins = np.logspace(np.log10(mc_energies_1.min()), np.log10(mc_energies_1.max()), 50)

# Create histogram
counts, bin_edges = np.histogram(mc_energies_1, bins=bins)
counts_2, bin_edges_2 = np.histogram(mc_energies_2, bins=bins)
# Calculate bin centers
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

# Plot the histogram
plt.figure(figsize=(8, 6))
plt.loglog(bin_centers, counts, label='MC Energy Distribution for proton simulation', color='red')
plt.loglog(bin_centers, counts_2, color='black', label = 'MC Energy Distribution for muon simulation')
plt.xlabel('Muon Energy (TeV)')
plt.ylabel('Number of Events')
plt.title('MC Energy Spectrum')
plt.legend()
plt.show()



In [ ]:
# Define a power-law function
def power_law(E, A, gamma):
    return A * E**-gamma

# Initial guesses for A and gamma
initial_guess = [1e3, 2]

# Perform the fit
popt, pcov = curve_fit(power_law, bin_centers, counts, p0=initial_guess)

A_fit, gamma_fit = popt
print(f"Fitted parameters: A = {A_fit}, gamma = {gamma_fit}")

# Plot the fit
plt.figure(figsize=(8, 6))
plt.loglog(bin_centers, counts, 'b.', label='MC Data')
plt.loglog(bin_centers, power_law(bin_centers, *popt), 'r-', label=f'Fit: $E^{{-{gamma_fit:.2f}}}$')
plt.xlabel('Muon Energy (TeV)')
plt.ylabel('Number of Events')
plt.title('Fitted MC Energy Spectrum')
plt.legend()
plt.show()

In [ ]:
# def dF_dE(data,zenith):
#     flux = data**-2.7 * (1/(1 + 1.1 * data * 1000 * np.cos(zenith)/115) + 0.054/(1 + 1.1 * data * 1000 * np.cos(zenith)/850))
#     return flux


def dPhi_dE_calc(energy, zenith):
    """ Calculate the muon flux at a given energy and zenith angle """
    # Calculate the cosine of the zenith angle
    cos_theta = np.cos(np.deg2rad(zenith))
    
    # Formula components
    term1 = 1 / (1 + (1.1 * energy * cos_theta) / 115)
    term2 = 0.054 / (1 + (1.1 * energy * cos_theta) / 850)
    
    # Calculate the flux
    flux = 1.4e3 * energy**-2.7 * (term1 + term2)
    
    return flux

# Energy range in GeV
E_mu = np.logspace(np.log10(10), np.log10(1000), 500)  # From 10 GeV to 1 TeV



In [ ]:
# Normalizing the flux distribution
E_mu = np.logspace(np.log10(10), np.log10(1000), 500)  # From 10 GeV to 1 TeV
dPhi_dE_normalized = dPhi_dE_calc(E_mu,10) / np.max(dPhi_dE_calc(E_mu,10))

# Creating a normalized power-law spectrum E^-2.7
power_law = E_mu**-2.7
power_law_normalized = power_law / np.max(power_law)

# Plotting both normalized distributions
plt.figure(figsize=(10, 6))
plt.plot(E_mu, dPhi_dE_normalized, label="Normalized Gaug Flux")
plt.plot(E_mu, power_law_normalized, label="Normalized $E^{-2.7}$ Spectrum", linestyle="--")
plt.xscale("log")
plt.yscale("log")
plt.xlabel(r"Energy $E_\mu$ [GeV]")
plt.ylabel("Normalized Flux")
plt.title("Normalized Muon Flux and Power-Law Spectrum")
plt.grid(True, which="both", linestyle="--", linewidth=0.5)
plt.legend()
plt.show()

# Proton sim reweigth

In [ ]:
data = df_sim_protons_combined.copy()
#data = df_sim_00046_align.copy()

# 2. Define Energy Bins
E_min = 0.1
E_max = 1
num_bins = 50  # Adjust as needed
bins = np.logspace(np.log10(E_min), np.log10(E_max), num=num_bins + 1)
#bins = np.linspace(E_min, E_max, num=num_bins + 1)
# 3. Bin the Data
data['energy_bin'] = pd.cut(data['mc_energy'], bins=bins, labels=False, include_lowest=True)
data.dropna(subset=['energy_bin'], inplace=True)

In [ ]:
# 4. Calculate Current Distribution
current_counts, _ = np.histogram(data['mc_energy'], bins=bins)
current_counts = np.where(current_counts == 0, 1, current_counts)  # Avoid division by zero

# 5. Define Desired Distribution (E^-2.7)
bin_centers = np.sqrt(bins[:-1] * bins[1:])  # Geometric mean for log bins
desired_counts = bin_centers**-2.7
#desired_counts = dPhi_dE_calc(bin_centers, 10)
desired_counts *= np.sum(current_counts) / np.sum(desired_counts)  # Normalize to total events

# 6. Compute Weights Per Bin
weights_per_bin = (desired_counts / current_counts)
print(weights_per_bin)
# 7. Assign Weights to Events


data['weight'] = data['energy_bin'].apply(lambda x: weights_per_bin[x])

# 8. Validate the Weighted Distribution
weighted_counts = current_counts * weights_per_bin

plt.figure(figsize=(10, 6))
plt.loglog(bin_centers, current_counts, label='Original Distribution', alpha=0.7)
plt.loglog(bin_centers, weighted_counts, label='Weighted Distribution (E^-2.7)', alpha=0.7)
plt.loglog(bin_centers, desired_counts, label='Desired E^-2.7 Spectrum', linestyle='--')
plt.xlabel('Muon Energy (TeV)')
plt.ylabel('Number of Events')
plt.title('Reweighting to Achieve E^-2.7 Energy Spectrum')
plt.legend()
plt.grid(True, which="both", ls="--")
plt.show()

# Additional Histogram Comparison
plt.figure(figsize=(10, 6))
plt.hist(data['mc_energy'], bins=bins, histtype='step', label='Original', color='blue')
plt.hist(data['mc_energy'], bins=bins, weights=data['weight'], histtype='step', label='Weighted (E^-2.7)', color='red')
plt.loglog(bin_centers, desired_counts, label='Desired E^-2.7 Spectrum', linestyle='--', color='green')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Muon Energy (TeV)')
plt.ylabel('Number of Events')
plt.title('Histogram Comparison: Original vs. Weighted')
plt.legend()
plt.grid(True, which="both", ls="--")
plt.show()

# 9. Verify Normalization
total_original = np.sum(current_counts)
total_desired = np.sum(desired_counts)
total_weighted = np.sum(data['weight'])

print(f"Total Original Counts: {total_original}")
print(f"Total Desired Counts: {total_desired:.2f}")
print(f"Total Weighted Counts: {total_weighted:.2f}")

In [ ]:
bins

In [ ]:
plt.figure(figsize=(10, 7.5))
z,x,c = plt.hist(data['ring_radius'][(data['ring_radius'] > 1) & (data['ring_radius'] < 1.235)], weights=data['weight'][(data['ring_radius'] > 1) & (data['ring_radius'] < 1.235)], bins=20, histtype='step', label='Weighted Muon Proton Sim', color='blue',density=True)
z,x,c = plt.hist(df_winter_data['ring_radius'][(df_winter_data['ring_radius'] > 1) & (df_winter_data['ring_radius'] < 1.235)], bins=20, histtype='step', label='Data', color='black',density=True)
z,x,c = plt.hist(df_sim_00046_align['ring_radius'][(df_sim_00046_align['ring_radius'] > 1) & (df_sim_00046_align['ring_radius'] < 1.235)], weights=reweight_gaug(df_sim_00046_align, 10)[(df_sim_00046_align['ring_radius'] > 1) & (df_sim_00046_align['ring_radius'] < 1.235)], bins=20, histtype='step', label='Weighted Muon Sim v1', color='red', density=True)
plt.legend()

In [ ]:
plt.figure(figsize=(10, 7.5))
z,x,c = plt.hist(data['ring_radius'], weights=data['weight'], bins=20, histtype='step', label='Weighted Muon Proton Sim', color='blue',density=True)
z,x,c = plt.hist(df_winter_data['ring_radius'], weights=reweight_gaug(df_winter_data, 10, True), bins=20, histtype='step', label='Data', color='black',density=True)
z,x,c = plt.hist(df_sim_00046_align['ring_radius'], weights=reweight_gaug(df_sim_00046_align, 10), bins=20, histtype='step', label='Weighted Muon Sim v1', color='red', density=True)
plt.legend()

In [ ]:
plt.figure(figsize=(10, 7.5))
z,x,c = plt.hist(df_sim_protons_combined['ring_radius'][(df_sim_protons_combined['ring_radius'] > 1) & (df_sim_protons_combined['ring_radius'] < 1.235)], weights=data['weight'][(df_sim_protons_combined['ring_radius'] > 1) & (df_sim_protons_combined['ring_radius'] < 1.235)], bins=20, histtype='step', label='Weighted Proton Sim', color='blue',density=True)
z,x,c = plt.hist(df_sim_00046_align['ring_radius'][(df_sim_00046_align['ring_radius'] > 1) & (df_sim_00046_align['ring_radius'] < 1.235)], weights=reweight_gaug(df_sim_00046_align, 10)[(df_sim_00046_align['ring_radius'] > 1) & (df_sim_00046_align['ring_radius'] < 1.235)], bins=20, histtype='step', label='Weighted Muon Sim', color='red', density=True)
plt.legend()

In [ ]:
df_sim_00046_align

In [ ]:
df_sim_protons_combined[df_sim_protons_combined['mc_energy'] > 0.15]

In [ ]:
data = df_sim_protons_combined.copy()

E_min = 0.1
E_max = 1
num_bins = 50 

bins = np.logspace(np.log10(E_min), np.log10(E_max), num=num_bins + 1)
#bins = np.linspace(E_min, E_max, num=num_bins + 1)

data['energy_bin'] = pd.cut(data['mc_energy'], bins=bins, labels=False, include_lowest=True)
data.dropna(subset=['energy_bin'], inplace=True)

In [ ]:
plt.figure(figsize=(10, 6))
z,x,c = plt.hist(data['mc_energy'], bins=bins, histtype='step', label='Original', color='blue', log=True, density=True)
plt.xlabel('Energy [TeV]')

In [ ]:
current_counts, _ = np.histogram(data['mc_energy'], bins=bins)
current_counts = np.where(current_counts == 0, 1, current_counts)  # Avoid division by zero
sum(current_counts)

In [ ]:
desired_counts

In [ ]:
bin_centers = np.sqrt(bins[:-1] * bins[1:])  # Geometric mean for log bins
desired_counts = bin_centers**-2.7
#desired_counts = dPhi_dE_calc(bin_centers, 10)
desired_counts *= np.sum(current_counts) / np.sum(desired_counts)  # Normalize to total events

plt.loglog(desired_counts)
plt.loglog(current_counts)
print(sum(current_counts))
print(sum(desired_counts))

In [ ]:
weights_per_bin = (desired_counts / current_counts)
weights_per_bin

# New way to reweight

In [ ]:
E_min, E_max = 0.15,0.9
protons_energy_cut = df_sim_protons_combined[(df_sim_protons_combined['mc_energy'] > E_min) & (df_sim_protons_combined['mc_energy'] < E_max)]
muons_energy_cut = df_sim_00046_align[(df_sim_00046_align['mc_energy'] > E_min) & (df_sim_00046_align['mc_energy'] < E_max)]
counts_hist,bins_hist,c = plt.hist(protons_energy_cut['mc_energy'], bins=50, histtype='step', label='Protons', color='blue', log=True, density=True)
z,x,c = plt.hist(muons_energy_cut['mc_energy'], weights=reweight_gaug(muons_energy_cut, 10), bins=50, histtype='step', label='Muons', color='red', log=True, density=True)

In [ ]:
# Calculate bin centers
bin_centers = (bins_hist[:-1] + bins_hist[1:]) / 2
nonzero = counts_hist > 0
bin_centers_nonzero = bin_centers[nonzero]
counts_nonzero = counts_hist[nonzero]
bin_centers, counts_hist

In [ ]:
def power_law(x, A, alpha):
    return A * x ** (-alpha)
# Example initial guesses for Power-law
initial_guesses = [max(counts_nonzero), 2.0]  # A and alpha

from scipy.optimize import curve_fit

# For Power-law
popt, pcov = curve_fit(power_law, bin_centers_nonzero, counts_nonzero, p0=initial_guesses)
# Extract fitted parameters
A_fit, alpha_fit = popt
print(f"Fitted Parameters:\nA = {A_fit:.2f}\nalpha = {alpha_fit:.2f}")

# Generate fitted curve
x_fit = np.linspace(bins_hist[0], bins_hist[-1], 1000)
y_fit = power_law(x_fit, *popt)

# Plot the histogram and the fitted function
plt.figure(figsize=(10, 6))
plt.bar(bin_centers, counts_hist, width=np.diff(bins_hist), alpha=0.6, label='Histogram Data', color='lightgreen', edgecolor='black', log=True)
plt.plot(x_fit, y_fit, 'r-', label='Fitted Power-law', linewidth=2)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Energy')
plt.ylabel('Counts')
plt.title('Log-Log Histogram with Fitted Power-law Function')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(11, 7))
z,x,c = plt.hist(protons_energy_cut['mc_energy'], bins=50, histtype='step', color='blue', log=True, density=True, label='Proton sim before reweighting')
z,x,c = plt.hist(protons_energy_cut['mc_energy'], weights=reweight_gaug(protons_energy_cut,10,(2.7-alpha_fit)), bins=50, histtype='step', color='red', log=True, density=True, label='Proton sim after reweighting')
z,x,c = plt.hist(muons_energy_cut['mc_energy'],weights=reweight_gaug(muons_energy_cut,10,0.7), bins=50, histtype='step', color='orange', log=True, density=True, label='Muon sim after reweighting')
plt.legend()
plt.xlabel('Energy [TeV]')

In [ ]:
plt.figure(figsize=(11, 7))
z,x,c = plt.hist(protons_energy_cut['ring_radius'],weights=reweight_gaug(protons_energy_cut,10,(2.7-alpha_fit)), bins=20, histtype='step', color='blue', log=True, density=True, label='Proton sim after reweighting')
z,x,c = plt.hist(muons_energy_cut['ring_radius'],weights=reweight_gaug(muons_energy_cut, 10, 0.7), bins=20, histtype='step', color='orange', log=True, density=True, label='Muon sim after reweighting')
z,x,c = plt.hist(df_winter_data['ring_radius'], weights=reweight_gaug(df_winter_data, 10, 10, True), bins=20, histtype='step', color='black', log=True, density=True, label='Data')
plt.legend()
plt.xlabel('Ring Radius')

In [ ]:
plt.figure(figsize=(11, 7))
plt.title("Energy range 0.15-0.9 TeV")
z,x,c = plt.hist(protons_energy_cut['ring_radius'][(protons_energy_cut['ring_radius'] > 1) & (protons_energy_cut['ring_radius'] < 1.235)],weights=reweight_gaug(protons_energy_cut,10,(2.7-alpha_fit))[(protons_energy_cut['ring_radius'] > 1) & (protons_energy_cut['ring_radius'] < 1.235)], bins=20, histtype='step', color='blue', log=True, density=True, label='Proton sim after reweighting')
z,x,c = plt.hist(muons_energy_cut['ring_radius'][(muons_energy_cut['ring_radius'] > 1) & (muons_energy_cut['ring_radius'] < 1.235)],weights=reweight_gaug(muons_energy_cut, 10, 0.7)[(muons_energy_cut['ring_radius'] > 1) & (muons_energy_cut['ring_radius'] < 1.235)], bins=20, histtype='step', color='orange', log=True, density=True, label='Muon sim after reweighting')
z,x,c = plt.hist(df_winter_data['ring_radius'][(df_winter_data['ring_radius'] > 1) & (df_winter_data['ring_radius'] < 1.235)], weights=reweight_gaug(df_winter_data, 10, 10, True)[(df_winter_data['ring_radius'] > 1) & (df_winter_data['ring_radius'] < 1.235)], bins=20, histtype='step', color='black', log=True, density=True, label='Data')
plt.legend()
plt.xlabel('Ring Radius')

In [ ]:
print(protons_energy_cut['mc_energy'].min())
print(protons_energy_cut['mc_energy'].max())
print(muons_energy_cut['mc_energy'].min())
print(muons_energy_cut['mc_energy'].max())

In [ ]:
print(protons_energy_cut['ring_radius'].min())
print(protons_energy_cut['ring_radius'].max())
print(muons_energy_cut['ring_radius'].min())
print(muons_energy_cut['ring_radius'].max())

In [ ]:
z,x,c = plt.hist(protons_energy_cut['mc_energy'], weights=protons_energy_cut['mc_energy']**(-0.71), bins=50, histtype='step', label='Protons', color='blue', log=True, density=True)
z,x,c = plt.hist(muons_energy_cut['mc_energy'], weights=reweight_gaug(muons_energy_cut,10), bins=50, histtype='step', label='Muons', color='red', log=True, density=True)


In [ ]:
plt.figure(figsize=(11, 7))
plt.title("Energy range 0.15-0.9 TeV")
z,x,c = plt.hist(protons_energy_cut['ring_radius'],weights=protons_energy_cut['mc_energy']**(-0.71), bins=20, histtype='step', color='blue', log=True, density=True, label='Proton sim after reweighting')
z,x,c = plt.hist(df_winter_data['ring_radius'], weights=reweight_gaug(df_winter_data, 10, 10, True), bins=20, histtype='step', color='black', log=True, density=True, label='Data')
plt.legend()
plt.xlabel('Ring Radius')